In [1]:
from sklearn.datasets import make_regression

# Generate regression toy data
n_samples = 1000
n_features = 5

X, y = make_regression(n_samples=n_samples, n_features=n_features, random_state=42)

# Split data into training and testing sets
train_ratio = 0.8
train_size = int(train_ratio * n_samples)

X_train = X[:train_size]
y_train = y[:train_size]

# Test data
X_test = X[train_size:]
y_test = y[train_size:]

In [2]:
from sklearn.model_selection import cross_val_score


def bo_params_generic(model, params, X_train, y_train):
    # Create the model instance with the specified parameters
    regressor = model(**params)
    
    # Assuming you have X_train, y_train defined for regression
    scores = cross_val_score(regressor, X_train, y_train, cv=10, scoring='neg_root_mean_squared_error')
    return -scores.mean()

In [9]:
all_models = {}

# Random Forest

**Feature selection**

**Hyperparameter optimalisatie**

In [4]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from bayes_opt import BayesianOptimization


params_ranges = {
    'splitter': (0, 1),  # Numeric mapping of string values: 'best' -> 0, 'random' -> 1
    'max_depth': (1, 20),
    'min_samples_leaf': (1, 10),
    'min_weight_fraction_leaf': (0.0, 0.5),
    'max_features': (0.1, 1),
    'max_leaf_nodes': (10, 100)
}

# Example usage with Decision Trees
model = DecisionTreeRegressor
dt_bo = BayesianOptimization(f=lambda splitter, max_depth, min_samples_leaf, min_weight_fraction_leaf,
                                    max_features, max_leaf_nodes: bo_params_generic(model, {
                                        'splitter': 'best' if splitter < 0.5 else 'random',
                                        'max_depth': round(max_depth),
                                        'min_samples_leaf': round(min_samples_leaf),
                                        'min_weight_fraction_leaf': min_weight_fraction_leaf,
                                        'max_features': max_features,
                                        'max_leaf_nodes': round(max_leaf_nodes)
                                    }, X_train, y_train),
                             pbounds=params_ranges)

results = dt_bo.maximize(n_iter=5, init_points=20)
params = dt_bo.max['params']

# Creating a model with the best hyperparameters
best_model = model(
    splitter='best' if params['splitter'] < 0.5 else 'random',
    max_depth=int(round(params['max_depth'])),
    min_samples_leaf=round(params['min_samples_leaf']),
    min_weight_fraction_leaf=params['min_weight_fraction_leaf'],
    max_features=params['max_features'],
    max_leaf_nodes=int(round(params['max_leaf_nodes']))
)

# Fit the model
best_model.fit(X_train, y_train)


|   iter    |  target   | max_depth | max_fe... | max_le... | min_sa... | min_we... | splitter  |
-------------------------------------------------------------------------------------------------
| 1         | 52.59     | 19.83     | 0.8834    | 16.19     | 6.064     | 0.4299    | 0.2575    |
| 2         | 63.49     | 5.666     | 0.1317    | 79.53     | 2.119     | 0.08336   | 0.5748    |
| 3         | 48.46     | 13.45     | 0.8559    | 28.66     | 3.64      | 0.1634    | 0.3241    |
| 4         | 51.66     | 3.851     | 0.645     | 73.52     | 7.612     | 0.1988    | 0.1656    |
| 5         | 64.51     | 9.96      | 0.9325    | 96.31     | 9.241     | 0.4936    | 0.547     |
| 6         | 63.8      | 4.662     | 0.1902    | 79.63     | 4.108     | 0.2016    | 0.952     |
| 7         | 55.03     | 6.409     | 0.669     | 55.28     | 9.796     | 0.263     | 0.2518    |
| 8         | 46.17     | 8.748     | 0.4099    | 35.67     | 9.691     | 0.05876   | 0.1181    |
| 9         | 61.38 

DecisionTreeRegressor(max_depth=10, max_features=0.9325423505773145,
                      max_leaf_nodes=96, min_samples_leaf=9,
                      min_weight_fraction_leaf=0.49359743828640723,
                      splitter='random')

In [ ]:
# Append model to list
all_models['Random forest'] = best_model